In [1]:
%matplotlib inline
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.utils import Sequence
from keras.utils import load_img
from keras.optimizers import RMSprop
from keras.optimizers import Adam


2023-01-25 11:39:01.385257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 11:39:03.781361: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /appl/cudnn/v8.3.2.44-prod-cuda-11.5/lib:/appl/cuda/11.6.0/lib64:/appl/python/3.10.7/lib:/appl/gcc/11.3.0-binutils-2.38/lib64:/appl/gcc/11.3.0-binutils-2.38/lib:/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2023-01-25 11:39:03.781719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so

In [2]:
df = pd.read_csv('black_background_500x500.csv')
train_df = df[df['ImagePath'].str.contains("train")]
test_df = df[df['ImagePath'].str.contains("test")]
valid_df = df[df['ImagePath'].str.contains("valid")]

input_size = 500

In [3]:
def load_df_to_list(the_df, i_s):
    the_df = the_df.reset_index()
    x_list = np.zeros((len(the_df), i_s, i_s, 1))
    y_list = np.zeros((len(the_df), 3))
    for i, row in the_df.iterrows():
        data = load_img(path = row['ImagePath'], color_mode = "grayscale")
        data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
        data /= 255
        x_list[i] = np.asarray(data)
        y_list[i] = row[['phi1','PHI', 'phi2']].values
    
    return x_list, y_list

In [4]:
# ImagePath,image_name,phi1,PHI,phi2
train_x_list,train_y_list = load_df_to_list(train_df, input_size)
test_x_list,test_y_list   = load_df_to_list(test_df, input_size)
valid_x_list,valid_y_list = load_df_to_list(valid_df, input_size)

In [5]:
train_x_list.shape

(8750, 500, 500, 1)

In [6]:
type(train_x_list)

numpy.ndarray

In [7]:
inputs = keras.Input(shape=(input_size, input_size, 1))
x = layers.Conv2D(filters=4, kernel_size=5, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Conv2D(filters=4, kernel_size=5, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Conv2D(filters=4, kernel_size=5, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(3)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

2023-01-25 11:40:39.854078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 11:40:40.753686: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14610 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:af:00.0, compute capability: 7.0


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 500, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 496, 496, 4)       104       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 4)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 120, 4)       404       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 4)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 4)         404   

In [8]:
import keras.backend as K
  
def abs_loss_function(y_true, y_pred):   
    abs_diff = K.abs(y_true - y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(ones - abs_diff )   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
    return tf.math.reduce_mean(minimum_from_two, axis=-1)

In [9]:
model.compile(optimizer = RMSprop(learning_rate=0.001),
              loss = abs_loss_function, 
              metrics = [abs_loss_function])

In [10]:
history = model.fit(x=train_x_list,
                    y=train_y_list,
                    batch_size=32,
                    epochs=5,
                    validation_split=0.1,
                    verbose=1)

2023-01-25 11:40:55.148645: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 7875000000 exceeds 10% of free system memory.
2023-01-25 11:41:03.867764: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 7875000000 exceeds 10% of free system memory.


Epoch 1/5


2023-01-25 11:41:11.883017: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-01-25 11:41:13.071525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8302


247/247 [==============================] - 14s 34ms/step - loss: 0.2476 - abs_loss_function: 0.2476 - val_loss: 0.2495 - val_abs_loss_function: 0.2495
Epoch 2/5
247/247 [==============================] - 8s 32ms/step - loss: 0.2473 - abs_loss_function: 0.2473 - val_loss: 0.2494 - val_abs_loss_function: 0.2494
Epoch 3/5
247/247 [==============================] - 7s 28ms/step - loss: 0.2453 - abs_loss_function: 0.2453 - val_loss: 0.2421 - val_abs_loss_function: 0.2421
Epoch 4/5
247/247 [==============================] - 7s 27ms/step - loss: 0.2410 - abs_loss_function: 0.2410 - val_loss: 0.2422 - val_abs_loss_function: 0.2422
Epoch 5/5
247/247 [==============================] - 7s 27ms/step - loss: 0.2373 - abs_loss_function: 0.2373 - val_loss: 0.2494 - val_abs_loss_function: 0.2494
